# Inimese-sekkumisega töövoog Microsoft Agent Frameworkiga

## 🎯 Õpieesmärgid

Selles märkmikus õpid, kuidas rakendada **inimese-sekkumisega** töövooge, kasutades Microsoft Agent Frameworki `RequestInfoExecutor`-it. See võimas muster võimaldab AI-töövoogudel peatuda, et koguda inimeste sisendit, muutes agendid interaktiivseks ja andes inimestele kontrolli oluliste otsuste üle.

## 🔄 Mis on inimese-sekkumine?

**Inimese-sekkumine (HITL)** on disainimuster, kus AI-agendid peatavad täitmise, et küsida inimeste sisendit enne jätkamist. See on oluline järgmistel juhtudel:

- ✅ **Olulised otsused** - Hankige inimeste heakskiit enne tähtsate toimingute tegemist
- ✅ **Ebaselged olukorrad** - Laske inimestel selgitada, kui AI on ebakindel
- ✅ **Kasutaja eelistused** - Küsige kasutajatelt, millist valikut nad eelistavad
- ✅ **Vastavus ja ohutus** - Tagage inimeste järelevalve reguleeritud toimingute puhul
- ✅ **Interaktiivsed kogemused** - Looge vestlusagendid, mis reageerivad kasutaja sisendile

## 🏗️ Kuidas see toimib Microsoft Agent Frameworkis

Raamistik pakub HITL-i jaoks kolme peamist komponenti:

1. **`RequestInfoExecutor`** - Eriline täitja, mis peatab töövoo ja väljastab `RequestInfoEvent`-i
2. **`RequestInfoMessage`** - Tüüpidega päringute põhiklass, mis saadetakse inimestele
3. **`RequestResponse`** - Seob inimeste vastused algsete päringutega, kasutades `request_id`

**Töövoo muster:**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 Meie näide: Hotelli broneerimine kasutaja kinnitusega

Me arendame tingimuslikku töövoogu, lisades inimeste kinnituse **enne**, kui soovitame alternatiivseid sihtkohti:

1. Kasutaja küsib sihtkohta (nt "Pariis")
2. `availability_agent` kontrollib, kas toad on saadaval
3. **Kui tube pole** → `confirmation_agent` küsib "Kas soovite näha alternatiive?"
4. Töövoog **peatub**, kasutades `RequestInfoExecutor`-it
5. **Inimene vastab** "jah" või "ei" konsooli sisendi kaudu
6. `decision_manager` suunab vastuse põhjal:
   - **Jah** → Näita alternatiivseid sihtkohti
   - **Ei** → Tühista broneerimistaotlus
7. Kuvatakse lõpptulemus

See näitab, kuidas anda kasutajatele kontroll agentide soovituste üle!

---

Alustame! 🚀


## Samm 1: Vajalike teekide importimine

Impordime standardseid Agent Frameworki komponente ja **inimese-sekkumisega seotud klassid**:
- `RequestInfoExecutor` - Täitja, mis peatab töövoo, et oodata inimese sisendit
- `RequestInfoEvent` - Sündmus, mis käivitatakse, kui oodatakse inimese sisendit
- `RequestInfoMessage` - Tüüpidega päringute põhiklass
- `RequestResponse` - Seostab inimese vastused päringutega
- `WorkflowOutputEvent` - Sündmus töövoo väljundite tuvastamiseks


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## Samm 2: Määra Pydantic mudelid struktureeritud väljundite jaoks

Need mudelid määratlevad **skeemi**, mille agendid tagastavad. Me säilitame kõik mudelid tingimusliku töövoo jaoks ja lisame:

**Uus inimese sekkumise jaoks:**
- `HumanFeedbackRequest` - `RequestInfoMessage` alamklass, mis määratleb inimeste jaoks saadetava päringu sisu
  - Sisaldab `prompt` (esitatav küsimus) ja `destination` (kontekst puuduvast linnast)


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## Samm 3: Loo hotellibroneerimise tööriist

Sama tööriist tingimusliku töövoo jaoks - kontrollib, kas sihtkohas on vabu tube.


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Samm 4: Tingimusfunktsioonide määratlemine suunamiseks

Meie inimese sekkumisega töövoo jaoks on vaja **nelja tingimusfunktsiooni**:

**Tingimuslikust töövoost:**
1. `has_availability_condition` - Suunab, kui hotellid ON saadaval
2. `no_availability_condition` - Suunab, kui hotellid EI ole saadaval

**Uued inimese sekkumise jaoks:**
3. `user_wants_alternatives_condition` - Suunab, kui kasutaja ütleb alternatiividele "jah"
4. `user_declines_alternatives_condition` - Suunab, kui kasutaja ütleb alternatiividele "ei"


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            wants_alternatives = "wants to see alternative" in msg_text or "want to see alternative" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                    <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{wants_alternatives}</strong>
                </div>
            """)
            )
            
            return wants_alternatives
    
    return False
def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            declined = "declined" in msg_text or "has declined" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{declined}</strong>
                </div>
            """)
            )
            
            return declined
    
    return False
print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## Samm 5: Loo otsuste halduri täitja

See on **inimese osalusega mustri tuum**! `DecisionManager` on kohandatud `Executor`, mis:

1. **Võtab vastu inimeste tagasisidet** `RequestResponse` objektide kaudu
2. **Töötleb kasutaja otsust** (jah/ei)
3. **Suunab töövoogu**, saates sõnumeid sobivatele agentidele

Peamised omadused:
- Kasutab `@handler` dekoraatorit, et muuta meetodid töövoo sammudeks
- Võtab vastu `RequestResponse[HumanFeedbackRequest, str]`, mis sisaldab nii algset päringut kui ka kasutaja vastust
- Annab lihtsaid "jah" või "ei" sõnumeid, mis käivitavad meie tingimusfunktsioonid


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and let the workflow route based on conditions.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        This handler just displays feedback and passes the RequestResponse through.
        The routing is done by condition functions on the edges.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Will route to alternative_agent
                </div>
            """)
            )
            # Create and send a message for the alternative_agent
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Will route to cancellation_agent
                </div>
            """)
            )
            # Create and send a message for the cancellation_agent
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input - treat as decline
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## Samm 6: Loo kohandatud kuvamise täitja

Sama kuvamise täitja tingimuslikust töövoost - annab lõplikud tulemused töövoo väljundina.


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## Samm 7: Laadi keskkonnamuutujad

Seadista LLM klient (GitHub Models, Azure OpenAI või OpenAI).


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## Samm 8: Loo AI agendid ja täiturid

Loome **kuus töövoo komponenti**:

**Agendid (pakitud AgentExecutorisse):**
1. **availability_agent** - Kontrollib hotellide saadavust tööriista abil
2. **confirmation_agent** - 🆕 Valmistab ette inimese kinnituspäringu
3. **alternative_agent** - Soovitab alternatiivseid linnu (kui kasutaja ütleb jah)
4. **booking_agent** - Julgustab broneerimist (kui toad on saadaval)
5. **cancellation_agent** - 🆕 Käsitleb tühistamissõnumit (kui kasutaja ütleb ei)

**Erilised täiturid:**
6. **request_info_executor** - 🆕 `RequestInfoExecutor`, mis peatab töövoo inimese sisendi jaoks
7. **decision_manager** - 🆕 Kohandatud täitur, mis suunab vastavalt inimese vastusele (juba ülal määratletud)


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## Samm 9: Töötage välja töövoog koos inimese sekkumisega

Nüüd loome töövoo graafiku koos **tingimusliku suunamisega**, mis sisaldab ka inimese sekkumise rada:

**Töövoo struktuur:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**Olulised ühendused:**
- `availability_agent → confirmation_agent` (kui tube pole saadaval)
- `confirmation_agent → prepare_human_request` (tüüp muudetakse)
- `prepare_human_request → request_info_executor` (paus inimese sekkumiseks)
- `request_info_executor → decision_manager` (alati - edastab RequestResponse'i)
- `decision_manager → alternative_agent` (kui kasutaja ütleb "jah")
- `decision_manager → cancellation_agent` (kui kasutaja ütleb "ei")
- `availability_agent → booking_agent` (kui toad on saadaval)
- Kõik rajad lõppevad `display_result` juures


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## Samm 10: Käivita testjuhtum 1 - Linn ILMA saadavuseta (Pariis koos inimkinnitusega)

See test näitab **täielikku inimese osalusega tsüklit**:

1. Hotelli päring Pariisis
2. availability_agent kontrollib → Pole tube
3. confirmation_agent loob inimesele suunatud küsimuse
4. request_info_executor **peatab töövoo** ja väljastab `RequestInfoEvent`
5. **Rakendus tuvastab sündmuse ja kuvab kasutajale konsoolis küsimuse**
6. Kasutaja sisestab "jah" või "ei"
7. Rakendus saadab vastuse `send_responses_streaming()` kaudu
8. decision_manager suunab vastuse põhjal
9. Lõpptulemus kuvatakse

**Oluline muster:**
- Kasuta `workflow.run_stream()` esimesel iteratsioonil
- Kasuta `workflow.send_responses_streaming(pending_responses)` järgmistel iteratsioonidel
- Kuula `RequestInfoEvent`, et tuvastada, millal on vaja inimese sisendit
- Kuula `WorkflowOutputEvent`, et tabada lõpptulemusi


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## Samm 11: Käivita testjuhtum 2 - Linn KOOS saadavusega (Stockholm - Inimese sekkumine pole vajalik)

See test näitab **otse teekonda**, kui toad on saadaval:

1. Hotelli päring Stockholmis
2. availability_agent kontrollib → Toad saadaval ✅
3. booking_agent soovitab broneerimist
4. display_result kuvab kinnituse
5. **Inimese sekkumine pole vajalik!**

Töövoog möödub täielikult inimese sekkumise teekonnast, kui toad on saadaval.


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## Olulised punktid ja parimad praktikad inimese kaasamiseks

### ✅ Mida olete õppinud:

#### 1. **RequestInfoExecutor muster**
Microsoft Agent Frameworki inimese kaasamise muster kasutab kolme põhikomponenti:
- `RequestInfoExecutor` - Peatab töövoo ja väljastab sündmusi
- `RequestInfoMessage` - Tüüpidega päringute põhiklass (laienda seda!)
- `RequestResponse` - Seob inimese vastused algsete päringutega

**Oluline arusaam:**
- `RequestInfoExecutor` ise sisendit ei kogu - see ainult peatab töövoo
- Teie rakenduse kood peab kuulama `RequestInfoEvent` sündmust ja koguma sisendi
- Peate kutsuma `send_responses_streaming()` koos sõnastikuga, mis seob `request_id` kasutaja vastusega

#### 2. **Voogesituse täitmise muster**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **Sündmustepõhine arhitektuur**
Kuulake konkreetseid sündmusi töövoo juhtimiseks:
- `RequestInfoEvent` - Vajalik on inimese sisend (töövoog peatatud)
- `WorkflowOutputEvent` - Lõpptulemus on saadaval (töövoog lõpetatud)
- `WorkflowStatusEvent` - Oleku muutused (IN_PROGRESS, IDLE_WITH_PENDING_REQUESTS jne)

#### 4. **Kohandatud täiturid koos @handleriga**
`DecisionManager` näitab, kuidas luua täitureid, mis:
- Kasutavad `@handler` dekoratsiooni, et muuta meetodid töövoo sammudeks
- Võtavad vastu tüüpidega sõnumeid (nt `RequestResponse[HumanFeedbackRequest, str]`)
- Suunavad töövoogu, saates sõnumeid teistele täituritele
- Juurdepääs kontekstile läbi `WorkflowContext`

#### 5. **Tingimuslik suunamine inimese otsuste põhjal**
Saate luua tingimusfunktsioone, mis hindavad inimese vastuseid:
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 Reaalsed rakendused:

1. **Kinnitamise töövood**
   - Hankige juhataja kinnitus enne kuludokumentide töötlemist
   - Nõudke inimese ülevaatust enne automaatsete e-kirjade saatmist
   - Kinnitage kõrge väärtusega tehingud enne täitmist

2. **Sisu modereerimine**
   - Märgistage kahtlane sisu inimese ülevaatuseks
   - Paluge moderaatoritel teha lõplik otsus keeruliste juhtumite puhul
   - Suunake inimesele, kui AI kindlus on madal

3. **Klienditeenindus**
   - Laske AI-l automaatselt käsitleda rutiinseid küsimusi
   - Suunake keerulised probleemid inimagentidele
   - Küsige kliendilt, kas ta soovib rääkida inimesega

4. **Andmetöötlus**
   - Paluge inimestel lahendada ebaselged andmekirjed
   - Kinnitage AI tõlgendused ebaselgetest dokumentidest
   - Laske kasutajatel valida mitme kehtiva tõlgenduse vahel

5. **Ohutuskriitilised süsteemid**
   - Nõudke inimese kinnitust enne pöördumatuid toiminguid
   - Hankige luba enne tundlike andmete juurde pääsemist
   - Kinnitage otsused reguleeritud tööstustes (tervishoid, rahandus)

6. **Interaktiivsed agendid**
   - Looge vestlusrobotid, mis esitavad järelküsimusi
   - Looge juhendid, mis aitavad kasutajaid keerulistes protsessides
   - Kujundage agendid, mis teevad inimestega samm-sammult koostööd

### 🔄 Võrdlus: tingimuslik töövoog vs inimese kaasamise töövoog

| Funktsioon | Tingimuslik töövoog | Inimese kaasamise töövoog |
|------------|---------------------|---------------------------|
| **Täitmine** | Üks `workflow.run()` | Tsükkel koos `run_stream()` + `send_responses_streaming()` |
| **Kasutaja sisend** | Puudub (täielikult automatiseeritud) | Interaktiivsed küsimused läbi `input()` või UI |
| **Komponendid** | Agendid + täiturid | + RequestInfoExecutor + DecisionManager |
| **Sündmused** | Ainult AgentExecutorResponse | RequestInfoEvent, WorkflowOutputEvent jne |
| **Peatamine** | Ei peatu | Töövoog peatub RequestInfoExecutor juures |
| **Inimese kontroll** | Puudub inimese kontroll | Inimesed teevad olulisi otsuseid |
| **Kasutusjuht** | Automatiseerimine | Koostöö ja järelevalve |

### 🚀 Täiustatud mustrid:

#### Mitmed inimese otsustuspunktid
Samasse töövoogu saab lisada mitu `RequestInfoExecutor` sõlme:
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### Aja ületamise käsitlemine
Rakendage aja ületamise mehhanism inimese vastuste jaoks:
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### Rikkalik UI integratsioon
`input()` asemel integreerige veebiliidese, Slacki, Teamsi jne-ga:
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### Tingimuslik inimese kaasamine
Küsige inimese sisendit ainult konkreetsetes olukordades:
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ Parimad praktikad:

1. **Alati laiendage RequestInfoMessage'i**
   - Tagab tüübikindluse ja valideerimise
   - Võimaldab rikkalikku konteksti UI renderdamiseks
   - Selgitab iga päringutüübi eesmärki

2. **Kasutage kirjeldavaid küsimusi**
   - Lisage kontekst, mida te küsite
   - Selgitage iga valiku tagajärgi
   - Hoidke küsimused lihtsad ja selged

3. **Käsitlege ootamatut sisendit**
   - Valideerige kasutaja vastuseid
   - Pakkuge vaikimisi väärtusi kehtetu sisendi korral
   - Andke selgeid veateateid

4. **Jälgige päringute ID-sid**
   - Kasutage korrelatsiooni request_id ja vastuste vahel
   - Ärge proovige käsitsi hallata olekut

5. **Kujundage mitteblokeerivaks**
   - Ärge blokeerige lõime sisendi ootamise ajal
   - Kasutage asünkroonseid mustreid
   - Toetage paralleelseid töövoo eksemplare

### 📚 Seotud kontseptsioonid:

- **Agendi vahevara** - Peatage agendi kõned (eelmine märkmik)
- **Töövoo oleku haldamine** - Säilitage töövoo olek jooksude vahel
- **Mitme agendi koostöö** - Kombineerige inimese kaasamine agentide meeskondadega
- **Sündmustepõhised arhitektuurid** - Looge reaktiivseid süsteeme sündmustega

---

### 🎓 Palju õnne!

Olete omandanud inimese kaasamise töövood Microsoft Agent Frameworkiga! Nüüd teate, kuidas:
- ✅ Peatada töövooge, et koguda inimese sisendit
- ✅ Kasutada RequestInfoExecutor ja RequestInfoMessage
- ✅ Käsitleda voogesituse täitmist sündmustega
- ✅ Luua kohandatud täitureid koos @handleriga
- ✅ Suunata töövooge inimese otsuste põhjal
- ✅ Luua interaktiivseid AI agente, mis teevad inimestega koostööd

**See on kriitiline muster usaldusväärsete ja kontrollitavate AI-süsteemide loomiseks!** 🚀



---

**Lahtiütlus**:  
See dokument on tõlgitud AI tõlketeenuse [Co-op Translator](https://github.com/Azure/co-op-translator) abil. Kuigi püüame tagada täpsust, palume arvestada, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Algne dokument selle algses keeles tuleks pidada autoriteetseks allikaks. Olulise teabe puhul soovitame kasutada professionaalset inimtõlget. Me ei vastuta arusaamatuste või valesti tõlgenduste eest, mis võivad tekkida selle tõlke kasutamise tõttu.
